In [5]:
import torch
from torch import nn
from torch import optim
from torch import autograd as grad
import torchvision
from torchvision.transforms import ToTensor


This code classifies cute dogs and not so cute cats

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convL1 = nn.Conv2d(in_channels=3,out_channels=64, kernel_size=(4,4), stride= (4,4))
        self.relu = nn.ReLU()
        self.poolL1 = nn.MaxPool2d((4,4), (4,4))
        #L3's input features size depends on the image size
        self.L3 = nn.Linear(65536, 2)

    def forward(self, x):
        x=self.convL1(x)
        print(x.shape)
        x=self.relu(x)
        x=self.poolL1(x)
        x=torch.flatten(x)
        x=self.L3(x)
        print(x.shape)
        return x

Train function section

In [7]:
def train_net(cnn, input):
    output = cnn.forward(input)  # Call forward on the instance
    print(output.shape)


In [8]:
if __name__ == '__main__':
    if torch.cuda.is_available() : dev = "cuda:0"
    else : dev = "cpu"
    dev = torch.device(dev)
    net=CNN().to(dev)
    
    fakeIn = torch.rand(3, 512, 512).to(dev)
    train_net(net, fakeIn)

torch.Size([64, 128, 128])
torch.Size([2])
torch.Size([2])
